## Authentication with OAuth2

Some of you may have already used APIs that require passing a specific key, to authorize the API calls. So the question naturally comes up: Why is that not sufficient for authentication?

Here are a few reasons:

* We often want our application to **act on behalf of a user** (e.g., retrieve the list of friends of a user on Facebook, and do some analysis on behalf of the user). OAuth allows for such delegation, without requiring the app to have access to the login credentials of the user.
* Acting on behalf of a user also allows the quota to be adjusted on a per-user basis, as opposed to a per-app basis. (So that the creators of very popular apps do not have to increase the quota for their own key)
* Users often want to give limited set of priviledges to the app (e.g., read only my profile, no posting).
* Users want to be able to selectively remove access for specific apps, without having to change the credentials for other apps.

So, how does OAuth achieves that?

## OAuth2 flows

Fundamentally, we have the following steps:

* The app sends the user to a login page. The login page asks the user whether the user really wants to grant these permissions to the app.
* The user logs in and grants the permissions. This generates an **authorization code** that the API sends back to the app (by **calling back a _redirect URL_**)
* The app uses the authorization code (which proves that the user has granted permissions), and calls the API, asking for an **access token**.
* The **access token** can then be used by the app to call the API on behalf of the user.

The picture below illustrates the OAuth2 flow:

<img src="https://assets.digitalocean.com/articles/oauth/abstract_flow.png">

Also these two tutorials explain in a simplified manner the details of the authentication process:
* https://www.digitalocean.com/community/tutorials/an-introduction-to-oauth-2
* http://aaronparecki.com/articles/2012/07/29/1/oauth2-simplified



### Running our own Python-based web server: Flask

As you can see from the above "flow", in OAuth2 authentication, we will need to have our own web server, to receive the answers that come back from the API. Python can do that using Flask, a very lightweight webserver. Below you can find a small example of a webserver that will run on your machine at port 5000. 

**Please note that we have configured this webserver to shutdown after a responding to a single call (we added the "stop\_server()" function call in call responses). In general, we do not do that, and we leave the server running.**

Once you run the call, visit http://yourIP:5000/ or http://yourIP:5000/testNYU to see the results. The server will stop immediately after, and the cell will stop being in "Busy" status.

In [ ]:
# In this cell, we configure our example web server. 
# We will show in the next cell how to configure this
# web server to handle the OAuth2 calls.

# Flask is a webserver
from flask import Flask, request

# We add a global variable that will be used to count the visitors to a specific URL
visitor_counter = 0
# Initialize the embedded web server
webserver = Flask("MyFirstWebServer")



# Go to http://<your IP>:5000/ to see the response
@webserver.route('/')
def hello_world():
    # stop_server()
    return 'Hello World!'

# Go to http://<your IP>:5000/testNYU to see the different message
@webserver.route('/testNYU')
def hello_nyu():
    # stop_server()
    global visitor_counter
    visitor_counter += 1
    return 'Hello! You are visitor #{i}'.format(i=visitor_counter)

def start_server():
    global visitor_counter
    visitor_counter = 0
    webserver.run(host='0.0.0.0', port=5000)
    return
    
def stop_server():
    shutdown_after_request = request.environ.get('werkzeug.server.shutdown')
    shutdown_after_request()
    return

start_server()

#### Exercise

Consult the notes in 1/D, and figure out how to run the server in the background.